In [1]:
import os
import gradio as gr
from langchain_community.document_loaders import DirectoryLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

/home/adam/Documents/adam/pdfMind/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 1. Configuration
DATA_PATH = "data"
LLM_MODEL_NAME = "mistral-nemo:latest"
EMBED_MODEL_NAME = "nomic-embed-text"
CHROMA_PATH = f"/home/adam/Documents/adam/pdfMind/chroma_db/{EMBED_MODEL_NAME}"

In [5]:
print("Creating embeddings and vector store (this may take a moment)...")
embeddings = OllamaEmbeddings(model=EMBED_MODEL_NAME)
llm = ChatOllama(model=LLM_MODEL_NAME)

Creating embeddings and vector store (this may take a moment)...


In [4]:
# 2. Load Documents
loader = DirectoryLoader(DATA_PATH, glob="*.pdf", loader_cls=PyMuPDFLoader)
documents = loader.load()

In [6]:
# 2. Fonction de nettoyage optimisée pour le Français
import re
def clean_french_text(text):
    text = text.replace('\xa0', ' ').replace('\n', ' ')
    text = text.replace("’", "'").replace("‘", "'")
    text = re.sub(r'(\w)-\s+(\w)', r'\1\2', text)
    text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text
cleaned_documents = []
for doc in documents:
    doc.page_content = clean_french_text(doc.page_content)
    if len(doc.page_content) > 20:
        cleaned_documents.append(doc)

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separators=["\n\n", "\n", ". ", " ", ""])
chunks = text_splitter.split_documents(cleaned_documents)
print(f"Split into {len(chunks)} chunks.")

Split into 305 chunks.


In [8]:
# Create Chroma vector store from documents
vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=CHROMA_PATH
)

In [9]:
retriever = vector_store.as_retriever(type="similarity", search_kwargs={"k": 5})

In [10]:
template = """
RÔLE :
Tu es un consultant expert en qualifications du bâtiment (Qualibat, RGE, Normes).
Ta mission est d'expliquer les documents techniques fournis de manière pédagogique, structurée et synthétique.

RÈGLES DE RÉDACTION (À SUIVRE IMPÉRATIVEMENT) :

1. STRUCTURE VISUELLE :
   - Commence toujours par une phrase d'introduction qui pose le contexte global.
   - Utilise des **titres de sections** clairs pour séparer les thématiques.
   - Utilise systématiquement des listes à puces (•) pour énumérer les détails.

2. PÉDAGOGIE ET DÉTAILS :
   - Si la question porte sur une **nomenclature ou un code** : Décortique la logique (ex: 1er chiffre = Famille). Donne un exemple concret (comme le code 2111 ou autre présent dans le contexte) pour illustrer.
   - Si la question porte sur des **règles/normes** : Cite précisément les références (ex: NF X 46-010) et les durées de validité.
   - Mets en **gras** les termes techniques importants, les chiffres clés et les concepts définis.

3. TON :
   - Professionnel, instructif et précis.
   - Ne dis jamais "D'après le contexte", intègre l'information comme une connaissance établie.

4. CONTRAINTE : Si la réponse n'est pas dans le contexte, dis "Je ne sais pas". N'invente rien.

CONTEXTE DOCUMENTAIRE :
{context}

---
QUESTION : 
{question}

RÉPONSE STRUCTURÉE :
"""
prompt = ChatPromptTemplate.from_template(template)

In [11]:
def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)


In [12]:
response = rag_chain.invoke("Quelles sont les 9 Famille d'activités ?")
answer = response["answer"]
context_docs = response["context"]
print(answer)
print("*"*100)

for doc in context_docs:
    print(doc.page_content)
    print("*"*100)

Bienvenue dans ce document technique portant sur les qualifications du bâtiment. Nous allons découvrir ensemble les différentes familles d'activités liées au domaine du bâtiment.

Il existe **neuf familles** d'activités, chacune correspondant à un domaine spécifique de travaux. Voici une liste des neuf familles avec leur intitulé :

* Famille 1 : PRÉPARATION DU SITE ET INFRASTRUCTURE
* Famille 2 : STRUCTURE ET GROS ŒUVRE
* Famille 3 : ENVELOPPE EXTÉRIEURE
* Famille 4 : CLOS - DIVISIONS - AMÉNAGEMENTS
* Famille 5 : ÉNERGIES ET FLUIDES
* Famille 6 : FINITIONS
* Famille 7 : ISOLATION THERMIQUE - ACOUSTIQUE - FRIGORIFIQUE
* Famille 8 : PERFORMANCE ÉNERGÉTIQUE
* Famille 9 : AGENCEMENT ET AMÉNAGEMENT

Chacune de ces familles regroupent des activités spécifiques liées au domaine du bâtiment. Par exemple, la Famille 2 concerne la structure et le gros œuvre, incluant notamment l'activité de maçonnerie et béton armé, tandis que la Famille 9 se concentre sur l'agencement et l'aménagement des loca

In [13]:
for dc in context_docs:
    print(f"{dc.page_content[:100]}\n{dc.metadata.get("page", "")}\n{"*"*100}")

Famille 1 | PRÉPARATION DU SITE ET INFRASTRUCTURE Famille 2 | STRUCTURE ET GROS ŒUVRE Famille 3 | EN
6
****************************************************************************************************
FAMILLE 2 STRUCTURE ET GROS ŒUVRE 2 1 2 3 4 5 6 7 8 9 2 15 Activité 21 | MAÇONNERIE ET BÉTON ARMÉ Sp
14
****************************************************************************************************
. (2) Sont concernées les 15 catégories de travaux (sur 19), du décret du 03 juin 2020. *E.C. : exig
76
****************************************************************************************************
9 FAMILLE 9 AGENCEMENT ET AMÉNAGEMENT 1 2 3 4 5 6 7 8 9 83 Activité 91 | AGENCEMENT ET AMÉNAGEMENT D
82
****************************************************************************************************
Activité 45 | FERMETURES ET STORES Spécialité 451 FOURNITURE ET POSE DE VOLETS, STORES, PORTAILS, RI
48
*****************************************************************************

In [14]:
response = rag_chain.invoke("Quels travaux nécessitent une certification Amiante ?")
answer = response["answer"]
context_docs = response["context"]
print(answer)
print("*"*100)

for doc in context_docs:
    print(doc.page_content)
    print("*"*100)

Les travaux nécessitant une certification amiante sont ceux concernant le traitement de l'amiante, c'est-à-dire les travaux de retrait ou d'encapsulage d'amiante et de matériaux, d'équipements et de matériels ou d'articles en contenant. Cette activité est règlementée et ne peut être réalisée que par des entreprises titulaires de la certification amiante.

références normatives :
- Normes NF X 46-010 et NF X 46-011
- Arrêté interministériel du 14 décembre 2012 fixant les conditions de certification des entreprises (JO du 02 février 2013)

Durée de la qualification :
La qualification est délivrée pour une durée ne pouvant excéder 4 ans pour une qualification quadriennale, 2 ans pour une probatoire et 5 ans pour une certification amiante. Le certificat est renouvelé tous les ans à partir d'un questionnaire de suivi obligatoire permettant la vérification et l'actualisation des informations initialement fournies.

**En résumé**, seule
********************************************************

In [15]:
response = rag_chain.invoke("Comment décrypter le code à 4 chiffres ?")
answer = response["answer"]
context_docs = response["context"]
print(answer)
print("*"*100)

for doc in context_docs:
    print(doc.page_content)
    print("*"*100)

Le document technique fournie explique comment décrypter le code qualification qui est composé de 4 chiffres. Chaque chiffre correspond à une information différente :

• Le premier chiffre représente la famille de travaux : dans ce cas, il s'agit des menuiseries extérieures.
• Le deuxième chiffre indique la sous-famille de travaux. Pour les menuiseries extérieures, il peut s'agir de la fourniture et pose de menuiseries extérieures en maison individuelle ou petit collectif (351), ou bien dans tout type de bâtiment (352).
• Le troisième chiffre précise le niveau de technicité requis pour l'activité. Par exemple, pour la fourniture et pose de menuiseries extérieures en maison individuelle ou petit collectif, il peut s'agir d'une technicité courante (1) ou confirmée (2).
• Le quatrième chiffre correspond à une mention spécifique. Dans le cas des qualifications 3511 et 3512, la mention "RGE" est possible.

Exemple concret : le code 3511 correspond à la fourniture et pose de menuiseries exté

In [16]:
response = rag_chain.invoke("Quelles sont les activités de la Famille 5 ?")
answer = response["answer"]
context_docs = response["context"]
print(answer)
print("*"*100)

for doc in context_docs:
    print(doc.page_content)
    print("*"*100)

La Famille 5 concerne l'ensemble des activités liées à la performance énergétique du bâtiment. Voici une liste détaillée de ces activités :

• Activité 51 | ÉTUDES, CONSEILS ET MAÎTRISE D’ŒUVRE EN ÉNERGIE
Qualification 511 Études, conseils et maîtrise d'œuvre en énergie - Bâtiments basse consommation (BBC) - Résidentiel et Tertiaire Mention “RGE“ possible

• Activité 52 | INSTALLATIONS ÉLECTRIQUES ET AUTRES ÉNERGÉTIQUES
Qualification 521 Installations électriques générales, spéciales et de protection Qualification 522 Installations photovoltaïques et éoliennes Qualification 523 Chauffage, ventilation, climatisation, chauffe-eau et cuisinières à gaz Qualification 524 Réfrigération, froid et surgelage Qualification 525 Cogénération Mention “RGE“ possible

• Activité 53 | INSTALLATIONS DE PRODUCTION D’ÉNERGIE ET DE CHAUFFAGE
Qualification 531 Chaudière à condensation et micro-cogénération gaz naturel, fioul ou bois Certification Qualibois possible Qualification 532 Pompe à chaleur air-air